# US Visas

In [19]:
#Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Input which data files are available in the
from subprocess import check_output

df = pd.read_csv('./us_perm_visas.csv')

df.info()

df.head()

#Show Entries
print('Number of Entries:', len(df))
print('Number of Columns:', len(df.columns))

sum(df.count() < 150000)

df['employer_city'].value_counts().head(10)

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden